In [ ]:
import pandas as pd
import re
import numpy as np
from config import config
from datetime import datetime
import requests
import csv
import os
import WRIC_preprocessing as wric
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)


### Example Usage

In [ ]:
R1_metadata, R2_metadata, df_room1, df_room2 = wric.preprocess_WRIC_file("C:\Documents\WRIC_example_data\Results_1m_copy_anonymised.txt", code="id+comment", path_to_save=None) #path_to_save="C:\Documents\WRIC_example_data"
display(df_room1)

# do not include discrepancy check standard and explain that without calibration gases no realistic values (only do for actual person values)
#check_discrepancies(df_room2, threshold=0.05)

In [ ]:
preprocess_WRIC_files('id.csv', 'upload')

In [ ]:
with open('./example_data/note.txt', 'r') as file:
    lines = file.readlines()

if not lines[-1].endswith('\n'):
    with open('./example_data/note.txt', 'a') as file:
        file.write('\n')

## Extracting protocol for all people 

In [ ]:
folder_path = "D:/Simon_CIRCLE/WRIC/processed"
protocol_dict = {"normal" : 0, "sleep" : 1, "eat" : 2, "active" : 3, "ree" : 4}

# choose the protocol you want (takes first) and number, if there are multiple specify the occurence (@Nina: start counting at 1!)
def tmp_func_name(folder_path, protocol, occurence = 1, add_start = 0, add_end = 0):
    # add_start, add_end in minutes
    wric_files = [f for f in os.listdir(folder_path) if f.endswith("_data.csv")]
    try:
        protocol_num = protocol_dict[protocol]
    except:
        print("ERROR: Please provide a valid protocol instance: normal, sleep, eat, active, ree")
        return
    for file in wric_files:
        print(file)
        df = pd.read_csv(folder_path +"/" + file)
        
        is_protocol = df["protocol"] == protocol_num
        transitions = is_protocol & (~is_protocol.shift(fill_value=False))
        occurence_index = transitions[transitions].index[occurence-1]
        start_datetime = df.loc[occurence_index, "datetime"]
        
        end_transitions = (~is_protocol) & is_protocol.shift(fill_value=False)
        try:
            end_index = end_transitions[end_transitions].index[occurence -1]
            end_datetime = df.loc[end_index, "datetime"]
        except IndexError:
            end_datetime = None
        
        start = pd.to_datetime(start_datetime) - pd.Timedelta(minutes=add_start)
        end = pd.to_datetime(end_datetime) + pd.Timedelta(minutes=add_end)
        
        # Check if start/end is earlier/later than the earliest/latest datetime in the DataFrame
        if start < pd.to_datetime(df["datetime"].min()):
            print(f"Warning: Start time {start} is earlier than the earliest data point. Using {df['datetime'].min()} instead.")
            start = pd.to_datetime(df["datetime"].min())
        if end > pd.to_datetime(df["datetime"].max()):
            print(f"Warning: End time {end} is later than the latest data point. Using {df['datetime'].max()} instead.")
            end = pd.to_datetime(df["datetime"].max())
          
        df = wric.cut_rows(df, start, end)
        display(df.head())
        if (set(df["protocol"].unique()) != {0, protocol_num}):
            print("WARNING: The time you specified includes other protocols than normal and", protocol, "Be aware of that for your analysis!")
            print(start, end)
            print(pd.isna(start), pd.isna(end))
        df.drop(columns=["relative_time[min]"])
           
        df = wric.add_relative_time(df)
        
        #display(df.head())
        
        # add/change reltive time
        
        #save
        
        
tmp_func_name(folder_path, "sleep", occurence=2)       

# output warning if changed to other protocol_num than 0 (interference)
# extrcat and save them as new DataFrames

In [ ]:
print("D:/Simon_CIRCLE/WRIC/01JJ_wric1min_v1_treat0.txt")

# Preprocessing all Data

In [ ]:
import WRIC_preprocessing as wric
import os

In [ ]:
folder1 = "/media/nina/SUNSHINE/Simon_CIRCLE/WRIC/Notes_Processed"
files_without_note = [file for file in os.listdir(folder1)]

print(files_without_note)

In [ ]:
wric_dict = {'01JJ_wric1min_v1_treat0.txt' : '01JJ_wric1min_v1_note_treat0.txt', 
    '01JJ_wric1min_v2_treat1.txt' : '01JJ_wric1min_v2_note_treat1.txt', 
    '02LK_v2_treat0_wric1min_04HH_v2_treat1.txt' : '02LK_v2_treat0_wric1min_04HH_v2_treat1_note.txt', 
    '02LK_wric1min_v1_treat1.txt' : '02LK_wric1min_v1_note_treat1.txt', 
    '03HA_v1_treat1_wric1min_04HH_v1_treat0.txt' : '03HA_v1_treat1_wric1min_04HH_v1_treat0_note.txt', 
    '03HA_wric1min_v2_treat0.txt' : '03HA_wric1min_v2_note_treat0.txt', 
    '05PM_wric1min_v1_treat0.txt' : '05PM_wric1min_v1_note_treat0.txt', 
    '05PM_wric1min_v2_treat1.txt' : '05PM_wric1min_v2_note_treat1.txt', 
    '06ML_v2_treat1_wric1min_09NQ_v1_treat1.txt' : '06ML_v2_treat1_wric1min_09NQ_v1_treat1_note.txt', 
    '06ML_wric1min_v1_treat0.txt' : '06ML_wric1min_v1_note_treat0.txt', 
    '07AB_v1_treat1_wric1min_08MG_v2_treat0.txt' : '07AB_v1_treat1_wric1min_08MG_v2_treat0_note.txt', 
    '07AB_wric1min_v2_treat0.txt' : '07AB_wric1min_v2_note_treat0.txt', 
    '08MG_wric1min_v1_treat1.txt' : '08MG_wric1min_v1_note_treat1.txt', 
    '09NQ_wric1min_v2_treat0.txt' : '09NQ_wric1min_v2_note_treat0.txt', 
    '10JK_wric1min_v1_treat0.txt' : '10JK_wric1min_v1_note_treat0.txt', 
    '10JK_wric1min_v2_treat1.txt' : '10JK_wric1min_v2_note_treat1.txt'}

In [ ]:
ids = ["01JJ", "02LK", "03HA", "04HH", "05PM", "06ML", "07AB", "08MG", "09NQ", "10JK"]
treatments = [0, 1]
base_folder = "/media/nina/SUNSHINE/Simon_CIRCLE/WRIC/"
note_base_folder = "/media/nina/SUNSHINE/Simon_CIRCLE/WRIC/Notes_Processed/"
path_to_save = "/media/nina/SUNSHINE/Simon_CIRCLE/WRIC/processed"

for filepath, notepath in wric_dict.items():
    print(filepath, notepath)
    R1_metadata, R2_metadata, df_room1, df_room2 = wric.preprocess_WRIC_file(base_folder+filepath, code="id+comment", path_to_save=path_to_save, notefilepath=note_base_folder+notepath)

# Playground for Note Integration

In [ ]:
R1_metadata, R2_metadata, df_room1, df_room2 = wric.preprocess_WRIC_file("C:/Documents/WRIC_example_data/Results_1m_copy_anonymised.txt") 


In [ ]:
# read the note file into a pandas Dataframe
notes_path = "C:/Documents/WRIC_example_data/Main_note_yyyymmddxxxx.txt"
notes_content = wric.open_file(notes_path)

lines = [line.strip().split('\t') for line in notes_content[2:]]

df_note = pd.DataFrame(lines[2:], columns=lines[0])
df_note = df_note.dropna()

# combine to datetime
df_note['datetime'] = pd.to_datetime(df_note['Date'] + ' ' + df_note['Time'], format='%m/%d/%y %H:%M:%S')
df_note = df_note.drop(columns=['Date', 'Time'])

In [ ]:
display(df_note.head())

In [ ]:
def save_dict(dict_protocol, participant, datetime, value):
    if participant is not None:
        dict_protocol[participant][datetime] = value
    else:
        dict_protocol[1][datetime] = value
        dict_protocol[2][datetime] = value
    return dict_protocol

In [ ]:
def update_protocol(df, protocol_list):
    """HELPER FUNCTION _ DO NOT USE! Update the protocol column in the given DataFrame based on the provided protocol list."""
    current_protocol = 0
    current_index = 0

    for index, row in df.iterrows():
        # While there are more timestamps and the current row's datetime is greater than or equal to the timestamp
        while (current_index < len(protocol_list) and 
               row['datetime'] >= protocol_list[current_index][0]):
            current_protocol = protocol_list[current_index][1]  # Update current protocol
            current_index += 1  # Move to the next timestamp

        df.at[index, 'protocol'] = current_protocol
        
    return df

In [ ]:


keywords_dict = {
    'sleeping': (["seng", "sleeping", "bed", "sove", "soeve"], 1),
    'eating': ([["start", "begin", "began"],["maaltid", "måltid", "eat", "meal", "food", "spis", "maal", "måd", "mad", "frokost", "morgenmad", "middag", "snack", "aftensmad"]], 2),
    'stop_sleeping' : (["vaagen", "vågen", "vaekket", "væk", "awake", "wake", "woken"], 0),
    'stop_anything': (["faerdig", "færdig", "stop", "end", "finished", "slut"], 0),
    'activity': ([["start", "begin", "began"], ["step", "exercise", "physicial activity", "active", "motion", "aktiv"]], 3),
    'ree_start': ([["start", "begin", "began"], ["REE"]], 4),
    # TODO: Cut based on start and end extracted from notes
    'end': (["ud", "exit", "out"], 0), #maybe as added safety check, check that it is the last/first note for that participant
    'start': (["ind i kammer", "enter", "ind"], 0)
}

time_pattern = r"([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5]\d"
dict_protocol = {1:{}, 2:{}}
for index, row in df_note.iterrows():
    participant = None
    if row["Comment"].startswith("1"):
        participant = 1
    elif row["Comment"].startswith("2"):
        participant = 2
    for category, (keywords, value) in keywords_dict.items():
        if isinstance(keywords[0], list):
            # Multi-group check: at least one keyword from each sublist must match
            if all(any(word.lower() in row['Comment'].lower() for word in group) for group in keywords):
                print("multi match", row["Comment"])
                # check if a different timestamp is written in the message and save the value there 
                # only checks first time stamp and only in format 6:45 or 06:45
                match = re.search(time_pattern, row['Comment'])
                if match:
                    time_str = match[0]
                    date_str = row['datetime'].date()
                    new_datetime = pd.Timestamp(datetime.combine(date_str, datetime.strptime(time_str, "%H:%M").time()))
                    dict_protocol = save_dict(dict_protocol, participant, new_datetime, value)
                else:
                    dict_protocol = save_dict(dict_protocol, participant, row["datetime"], value)
        else:
            # Single-group check: only one keyword needs to match
            if any(word.lower() in row['Comment'].lower() for word in keywords):
                match = re.search(time_pattern, row['Comment'])
                if match:
                    time_str = match[0]
                    date_str = row['datetime'].date()
                    new_datetime = pd.Timestamp(datetime.combine(date_str, datetime.strptime(time_str, "%H:%M").time()))
                    dict_protocol = save_dict(dict_protocol, participant, new_datetime, value)
                else:
                    dict_protocol = save_dict(dict_protocol, participant, row["datetime"], value)
            
protocol_list_1 = sorted(dict_protocol[1].items())
protocol_list_2 = sorted(dict_protocol[2].items())

update_protocol(df_room1, protocol_list_1)
update_protocol(df_room2, protocol_list_2)

print(protocol_list_1)
print(protocol_list_2)
            

In [ ]:
display(df_room1)
df_room1.to_csv("./tmp/df_room1.csv")

In [ ]:
a = "bæsrefg 9:33 dlsfgkbnwee 10:54 wkregf"
b = "kljbfdavg wergoi"
pattern = r"\b([01]?\d|2[0-3]):[0-5]\d\b"
match = re.search(pattern, b)

In [ ]:
print(bool(match))